In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

The 60% of images are used for training, and 20% are used for validation and rest of them are used for test. I added the kind.csv under /kaggle/input/fortest, which contained the bondary of these. For example, for 'airplane' label, first 0 to 436 images which are read are for training and 437-582 images are for validation, and 583-726 images are for test.

In this script, I used simple model, but if I modified a little, I could use IncetionV3, which could achive almost 99% accuracy in only several epochs. in this script which used simple model, I could reach 90% accurecy. 

Here are brief explanation of function
1. read_image_fn
   read file names of each label
2. read_img
   read actual images and labels accoring to filenames which read in read_image_fn and file counts of each label. Image size (139,139) are minimum size of InceptionV3 in case of using it.
3. read_size
   read kind.csv
4. model_create
   model creation. 
5. display_result
   display graphs for accuracy and loss.
6. main
   main routine, in this function, ImageDataGenerator is included to increase images. Also, only several lines are commented out/in and little changed, I can use InceptionV3 as base model.



In [ ]:
#coding:cp932
# 2021.04.05 14.17 Created
# 2021.04.08 10:36 Use own model insted of InceptionV3

import numpy as np
import pandas as pd
import os
from keras.preprocessing.image import load_img
from keras.utils.np_utils import to_categorical

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, Input, BatchNormalization
from keras.applications.inception_v3 import InceptionV3
from keras.layers import GlobalAveragePooling2D, MaxPooling2D
from keras.models import Model, load_model
from keras import layers, optimizers
from keras.preprocessing.image import img_to_array, load_img
import matplotlib.pyplot as plt
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

IMAGE_DIR="/kaggle/input/natural-images/data/natural_images/"
BASE_DIR=".\/"
kind_data = {'airplane':0 ,'car':1,'cat':2,'dog':3,'flower':4,'fruit':5,'motorbike':6,'person':7}
CATEGORY_SIZE=8

train_data = []
train_label = []
valid_data = []
valid_label = []

def read_image_fn():
    mydict = {}
    
    for i, animal in enumerate(kind_data.keys()):
        #print("animal.key=",animal)
        direc = IMAGE_DIR + animal
        files = os.listdir(direc)
        data = []
        for file in files:
            data.append(file)
       
        mydict[animal] = data
        
    return mydict

def read_img(mydict, size_data):
    global train_data, valid_data, train_label, valid_label
    
    kind_count = {'airplane':0 ,'car':0,'cat':0,'dog':0,'flower':0,'fruit':0,'motorbike':0,'person':0}
    
    for kind in mydict.keys():
        #print("kind=",kind)
        for fn in mydict[kind]:
            if kind_count[kind] <= int(size_data.loc[kind]['train']):
                kind_count[kind] += 1
                img = load_img(IMAGE_DIR + kind + "/" + fn, grayscale=False, target_size=(139,139))
                img_array = img_to_array(img)
                train_data.append(img_array)
                train_label.append(kind_data[kind])
            elif kind_count[kind] <= int(size_data.loc[kind]['valid']):
                kind_count[kind] += 1
                img = load_img(IMAGE_DIR + kind + "/" + fn, grayscale=False, target_size=(139,139))
                img_array = img_to_array(img)
                valid_data.append(img_array)
                valid_label.append(kind_data[kind])
            else:
                continue

    train_label = np.asarray(train_label)
    valid_label = np.asarray(valid_label)                
    train_label = to_categorical(train_label,CATEGORY_SIZE)
    valid_label = to_categorical(valid_label,CATEGORY_SIZE)
    train_data = np.asarray(train_data)
    train_data = train_data.astype('float32')/255.0    
    valid_data = np.asarray(valid_data)
    valid_data = valid_data.astype('float32')/255.0
    
def read_size():
    size_data = pd.read_csv("/kaggle/input/fortest/kind.csv", index_col=0)
    
    return size_data

def model_create():

    #base_model = keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_tensor=layers.Input(shape=(139,139,3)), input_shape=(139,139,3))
    #x = base_model.output
    
    visible = Input(shape=(139,139,3))
    x = Conv2D(64,kernel_size=3,activation='relu')(visible)
    #x = Conv2D(64,kernel_size=3,activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(32,kernel_size=3,activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D()(x)
    x = Conv2D(16,kernel_size=3,activation='relu')(x) 
    x = BatchNormalization()(x) 
    x = GlobalAveragePooling2D()(x) 
    x = Flatten()(x)
    x = Dense(1024,activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(CATEGORY_SIZE, activation='softmax')(x)
    model = Model(inputs=visible, outputs=predictions)

    #for layer in model.layers[:310]:
    #    layer.trainable = False
        
    model.summary()
    return model
    
def display_result(graph_viz):
    plt.figure(figsize=(10,10))
    plt.plot(graph_viz.history['accuracy'])
    plt.plot(graph_viz.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    plt.figure(figsize=(10,10))
    plt.plot(graph_viz.history['loss'])
    plt.plot(graph_viz.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'],loc='upper left')
    plt.show()
    
def main():
    global train_data, valid_data, train_label, valid_label
    
    size_data = read_size()
    mydict = read_image_fn()
    read_img(mydict, size_data)
    model = model_create()
    
    model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),metrics=['accuracy'])
    
    datagen = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        rotation_range=20,
        #width_shift_range=0.2,
        #height_shift_range=0.2,
        horizontal_flip=True,
        #vertical_flip=True,
        #zoom_range=[0.2,0.2],
        #brightness_range=[0.2,0.2]
        )
        
    datagen.fit(train_data)
    
    graph_viz = model.fit(x = train_data,y = train_label, batch_size=32, epochs=400, verbose=2,
                          validation_data = (valid_data, valid_label),shuffle=True)

    print("Save model")
    model.save("./my_model.h5")
    print("Finish save model")
    display_result(graph_viz)
    
if __name__ == "__main__":
    main()
    


Here is code for prediction, modified the above traing code. So the useless code are included. This read my_model.h5 which is produced in above code and usig the image files of 20% of total images.  Also this produces the prediction label and actual label in result.csv

In [ ]:
#coding:cp932
# 2021.04.05 14.17 Created

import numpy as np
import pandas as pd
#import glob
import os
from keras.preprocessing.image import load_img
from keras.utils.np_utils import to_categorical

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications.inception_v3 import InceptionV3
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D
from keras.models import Model
from keras import layers, optimizers
from keras.preprocessing.image import img_to_array, load_img

IMAGE_DIR="/kaggle/input/natural-images/data/natural_images/"
BASE_DIR="./"
kind_data = {'airplane':0 ,'car':1,'cat':2,'dog':3,'flower':4,'fruit':5,'motorbike':6,'person':7}
CATEGORY_SIZE=8

test_data = []
test_label = []


def read_image_fn():
    mydict = {}
    
    for i, animal in enumerate(kind_data.keys()):
        #print("animal.key=",animal)
        direc = IMAGE_DIR + animal
        files = os.listdir(direc)
        data = []
        for file in files:
            data.append(file)
       
        mydict[animal] = data
        
    #print(mydict)
    return mydict

def read_img(mydict, size_data):
    global test_data, test_label
    
    #print("saize=",str(len(mydict)))
    kind_count = {'airplane':0 ,'car':0,'cat':0,'dog':0,'flower':0,'fruit':0,'motorbike':0,'person':0}
    
    for kind in mydict.keys():
        #print("kind=",kind)
        for fn in mydict[kind]:
            #print("limit=", size_data.loc[kind]["test"])
            #print("fn=",fn)
            kind_count[kind] += 1
            if kind_count[kind] > int(size_data.loc[kind]['valid']) and kind_count[kind] <= int(size_data.loc[kind]['test']):
                img = load_img(IMAGE_DIR + kind + "/" + fn, grayscale=False, target_size=(139,139))
                img_array = img_to_array(img)
                test_data.append(img_array)
                test_label.append(kind_data[kind])
            else:
                continue

    test_label = np.asarray(test_label)               
    test_label = to_categorical(test_label,CATEGORY_SIZE)
    test_data = np.asarray(test_data)
    test_data = test_data.astype('float32')/255.0    
    
def read_size():
    size_data = pd.read_csv("/kaggle/input/fortest/kind.csv", index_col=0)
    #size_data.set_index('kind')
    #print(size_data)
    return size_data

def model_create():

    model = keras.models.load_model("./my_model.h5")
    return model
        
def main():
    global test_data, test_label
    
    size_data = read_size()
    mydict = read_image_fn()
    read_img(mydict, size_data)
    model = model_create()
    
    predictions = model.predict(test_data)
    
    for i in range(len(predictions)): 
        print("[",i,"]=", np.argmax(predictions[i]), "test_label=", np.argmax(test_label[i]))
        with open('result.csv', 'a') as f:
            f.writelines(str(np.argmax(predictions[i]))+","+str(np.argmax(test_label[i]))+"\n")
                          
if __name__ == "__main__":
    main()
    


This code is modified from above code. So useless code are included. This produved table for unmatched count between prediction and actual label. and from this result, "cat" and "dog" are difficult to recognized correctly for this simple model.

In [ ]:
#coding:cp932
# 2021.04.05 14.17 Created

import numpy as np
import pandas as pd
#import glob
import os
from keras.preprocessing.image import load_img
from keras.utils.np_utils import to_categorical

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications.inception_v3 import InceptionV3
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D
from keras.models import Model
from keras import layers, optimizers
from keras.preprocessing.image import img_to_array, load_img

kind_data = {'airplane':0 ,'car':1,'cat':2,'dog':3,'flower':4,'fruit':5,'motorbike':6,'person':7}
CATEGORY_SIZE=8
    
def read_size():
    size_data = pd.read_csv("result.csv", names=('pre','tst'))
    print(size_data)
    return size_data
        
def main():
   
    size_data = read_size()
    
    data_frame = pd.DataFrame()
    
    for my_key_i,my_value_i in kind_data.items():
        for my_key_j, my_value_j in kind_data.items():
            data_frame.loc[my_key_i, my_key_j] = 0
    
    print(data_frame)
    j = 0

    for i in range(len(size_data)):
        pre = int(size_data.iloc[i]['pre'])
        tst = int(size_data.iloc[i]['tst'])
        pre_key = [k for k, v in kind_data.items() if v == pre]
        tst_key = [k for k, v in kind_data.items() if v == tst]
        if size_data.iloc[i]['pre'] != size_data.iloc[i]['tst']:
            j += 1
            print(j, " ", "pre=",pre_key, "tst=", tst_key)
            data_frame.loc[pre_key,tst_key] += 1

    print(data_frame)
                          
if __name__ == "__main__":
    main()
    
